# Building Datasets From multiple Sources

```
1. remember, that
1. with markdown
1. you can make a numbered list
1. using all `1.`
```

renders as:

1. remember, that
1. with markdown
1. you can make a numbered list
1. using all `1.`

In [1]:
import pandas as pd

In [2]:
course_data_url = 'https://raw.githubusercontent.com/rhodyprog4ds/rhodyds/main/data/'

Today we're going to look at some data on NBA  (National Basketball Association) players.  

## Combining Multiple Tables

In [3]:
p18 = pd.read_csv(course_data_url +'2018-players.csv')
p19 = pd.read_csv(course_data_url +'2019-players.csv')

In [4]:
p18.head()

,TEAM_ID,PLAYER_ID,SEASON
0,1610612761,202695,2018
1,1610612761,1627783,2018
2,1610612761,201188,2018
3,1610612761,201980,2018
4,1610612761,200768,2018


### Stacking with Concat

We've seen one way of putting dataframes together with `concat` we used it when
{ref}`unpacking-jsons`. In that case, we stacked them side by side, now we want
to stack them vertically, which is the default of concat.

In [5]:
players_df = pd.concat([p18,p19])
players_df.head()

,TEAM_ID,PLAYER_ID,SEASON,PLAYER_NAME
0,1610612761,202695,2018,NaN
1,1610612761,1627783,2018,NaN
2,1610612761,201188,2018,NaN
3,1610612761,201980,2018,NaN
4,1610612761,200768,2018,NaN


This has the same columns, and we can see what happened more by looking at the
shape of each.

In [6]:
players_df.shape

(1374, 4)

In [7]:
p18.shape, p19.shape

((748, 3), (626, 4))

We can verify that the length of the new data frame is the sum of the original
two DataFrames.

In [8]:
assert len(p18) + len(p19) == len(players_df)

### Combining Data with Merge

What is we want to see which players changed teams from 2018 to 2019? We can do
this with `merge`. For `merge` we have to tell it a left DataFrame and a right
DataFrame and on what column to match them up.  

The left and right DataFrames will be used different ways, but any DataFrame
can be put in either position.  

For this case, we will use 2018 data as left ,and 2019 as right and then merge
`on='PLAYER_ID'`.

In [9]:
pd.merge(p18,p19,on='PLAYER_ID').head()

,TEAM_ID_x,PLAYER_ID,SEASON_x,PLAYER_NAME,TEAM_ID_y,SEASON_y
0,1610612761,202695,2018,Kawhi Leonard,1610612746,2019
1,1610612761,1627783,2018,Pascal Siakam,1610612761,2019
2,1610612761,201188,2018,Marc Gasol,1610612761,2019
3,1610612763,201188,2018,Marc Gasol,1610612761,2019
4,1610612761,201980,2018,Danny Green,1610612747,2019


Now, we get what we expect, but the column names have `_x` and `_y` on the end
(as a {term}`suffix`, appended to the original).  We'll add 2018 and 2019 respectively, separated with a `_`.

In [10]:
year_over_year = pd.merge(p18,p19,on='PLAYER_ID',suffixes=('_2018','_2019'))
year_over_year.head()

,TEAM_ID_2018,PLAYER_ID,SEASON_2018,PLAYER_NAME,TEAM_ID_2019,SEASON_2019
0,1610612761,202695,2018,Kawhi Leonard,1610612746,2019
1,1610612761,1627783,2018,Pascal Siakam,1610612761,2019
2,1610612761,201188,2018,Marc Gasol,1610612761,2019
3,1610612763,201188,2018,Marc Gasol,1610612761,2019
4,1610612761,201980,2018,Danny Green,1610612747,2019


Now that it's a little bit cleaner, we will examine how it works by looking at
the shape.

In [11]:
year_over_year.shape, p18.shape, p19.shape

((538, 6), (748, 3), (626, 4))

This kept only the players that played both years, with repetitions for each team they played on for each year.


We can check the calculation with set math (python `set` type has operations
  from math sets like intersect and set difference)

In [12]:
# player IDs for each year, no repeats
p19_u = set(p19['PLAYER_ID'])
p18_u = set(p18['PLAYER_ID'])

# player IDs that played both years
p1819 = p18_u.intersection(p19_u)

# teams per player per year
teams_per_p18 = p18['PLAYER_ID'].value_counts()
teams_per_p19 = p19['PLAYER_ID'].value_counts()

# total number of team-player combinations
# multiply number of teams each player played for in 18 by number of teams in 19
# then sum.  (most of of these are 1*1)
sum(teams_per_p19[p1819]* teams_per_p18[p1819])

/tmp/ipykernel_2712/3008611582.py:15: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  sum(teams_per_p19[p1819]* teams_per_p18[p1819])
/tmp/ipykernel_2712/3008611582.py:15: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  sum(teams_per_p19[p1819]* teams_per_p18[p1819])


538

We can also merge so that we keep all players on either team using `how='outer'`
the default value for `how` is inner, which takes the intersection (but with duplicates, does some extra things as we saw).  With `outer` it takes the union,
but with extra handling for the duplicates.

In [13]:
pd.merge(p18,p19,on='PLAYER_ID',suffixes=('_2018','_2019'),how='outer').shape

(927, 6)

It's the total of the rows we had before, plus the total number of player-teams
for players that only played in one of the two years.

In [14]:
#players tha tonly played in one year
o18 = p18_u.difference(p19_u)
o19 = p19_u.difference(p18_u)
# teams those players played for + the above 538
teams_per_p19[o19].sum() + teams_per_p18[o18].sum() + sum(teams_per_p19[p1819]* teams_per_p18[p1819])

/tmp/ipykernel_2712/955177081.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  teams_per_p19[o19].sum() + teams_per_p18[o18].sum() + sum(teams_per_p19[p1819]* teams_per_p18[p1819])
/tmp/ipykernel_2712/955177081.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  teams_per_p19[o19].sum() + teams_per_p18[o18].sum() + sum(teams_per_p19[p1819]* teams_per_p18[p1819])
/tmp/ipykernel_2712/955177081.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  teams_per_p19[o19].sum() + teams_per_p18[o18].sum() + sum(teams_per_p19[p1819]* teams_per_p18[p1819])
/tmp/ipykernel_2712/955177081.py:5: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  teams_per_p19[o19].sum() + teams_per_p18[o18].sum() + sum(teams_per_p19[p1819]* teams_per_

927

We can save this to a variable

In [15]:
year_over_year_outer =  pd.merge(p18,p19,on='PLAYER_ID',suffixes=('_2018','_2019'),how='outer')

then look at a few rows to see that it has indeed filled in with NaN in the
places where there wasn't a value.

In [16]:
year_over_year_outer.sample(10)

,TEAM_ID_2018,PLAYER_ID,SEASON_2018,PLAYER_NAME,TEAM_ID_2019,SEASON_2019
603,1.610613e+09,101162,2018.0,NaN,NaN,NaN
616,1.610613e+09,203463,2018.0,Ben McLemore,1.610613e+09,2019.0
68,1.610613e+09,1628470,2018.0,Torrey Craig,1.610613e+09,2019.0
740,1.610613e+09,1629226,2018.0,NaN,NaN,NaN
452,1.610613e+09,203121,2018.0,Darius Miller,1.610613e+09,2019.0
289,1.610613e+09,201587,2018.0,Nicolas Batum,1.610613e+09,2019.0
903,NaN,1629665,NaN,Jalen Lecque,1.610613e+09,2019.0
700,1.610613e+09,1627858,2018.0,Kyle Collinsworth,1.610613e+09,2019.0
639,1.610613e+09,1626150,2018.0,Andrew Harrison,1.610613e+09,2019.0
826,NaN,1629076,NaN,Tyler Cook,1.610613e+09,2019.0


```{note}
We can also tell that there are NaN beacuse it cast the year to float from int.
```

## Merge types, in detail

We can examine how these things work more visually with smaller DataFrames:

In [17]:
left = pd.DataFrame(
  {
    "key1": ["K0", "K0", "K1", "K2"],
    "key2": ["K0", "K1", "K0", "K1"],
    "A": ["A0", "A1", "A2", "A3"],
    "B": ["B0", "B1", "B2", "B3"],
  }
)


right = pd.DataFrame(
  {
    "key1": ["K0", "K1", "K1", "K2"],
    "key2": ["K0", "K0", "K0", "K0"],
    "C": ["C0", "C1", "C2", "C3"],
    "D": ["D0", "D1", "D2", "D3"],
  }
)


In [18]:
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [19]:
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


````{margin}
```{note}
inner is default, but we can state it to be more explicit
```
````

In [20]:
pd.merge(left, right, on=["key1", "key2"], how='inner')


,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [21]:
pd.merge(left, right, on=["key1", "key2"], how='outer' )

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K0,K1,A1,B1,NaN,NaN
2,K1,K0,A2,B2,C1,D1
3,K1,K0,A2,B2,C2,D2
4,K2,K1,A3,B3,NaN,NaN
5,K2,K0,NaN,NaN,C3,D3


## Duplicate Keys

In [22]:
left = pd.DataFrame({"A": [1, 2], "B": [2, 2]})

right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2]})

result = pd.merge(left, right, on="B", how="outer")

In [23]:
left

,A,B
0,1,2
1,2,2


In [24]:
right

,A,B
0,4,2
1,5,2
2,6,2


In [25]:
result

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


If we ask pandas to validate the merge with `validate` and a specific type of
merge, it will throw an error if that type of merge is not possible.

In [26]:
pd.merge(left, right,on='B',validate='one_to_one')

MergeError: Merge keys are not unique in either left or right dataset; not a one-to-one merge

```{admonition} Further Reading
The [pandas documentation](https://pandas.pydata.org/docs/user_guide/merging.html#checking-for-duplicate-keys) is a good place to read through their
exampels on how validate works. it's important to note the types of possible joins from the [beginning of the section](https://pandas.pydata.org/docs/user_guide/merging.html#brief-primer-on-merge-methods-relational-algebra)
```

## Questions at the End of Class


### how to remove certain columns in merges (i.e. the year columns in the basketball dataset since they are redundant)
````{toggle}
They can be dropped after (or before) using `drop()`

```{code-cell} ipython3
year_over_year_clean = year_over_year.drop(columns = ['SEASON_2018','SEASON_2019'])
year_over_year_clean.head()
```

````

### Can we merge as many dataframes as we would want?
```{toggle}
Merge is specifically defined for two Data Frames, but the result returns a
a DataFrame, so it could be passed to another merge.
```


### Is there no way to merge the left and right for the A and B data?
```{toggle}
All merges are possible, they just each give a different result.
```


### Can you use merging to check correlation between two different sets?
```{toggle}
You can use merging to check *overlap* between two DataFrames, for actual sets, it's probably best to represent them as `set` type objects and use set operations.

However, these type of operations are not actually correlations, which we'll learn a bit about in a few weeks when we talk about regression
```


### Is there any way to compare the team ids in both datasets so that it outputs the players that changed teams between those times?

````{toggle}
From the `year_over_year` DataFrame we made above, we can check which players changed teams.  Since players also can change teams *within* a season, this is a tricky question, but we could, for example look only for players that have no overlapping teams from 2018 to 2019.



````

In [27]:
change_check = lambda r: not(r['TEAM_ID_2018']== r['TEAM_ID_2019'])
year_over_year_clean['CHANGE_TEAM'] = year_over_year_clean.apply(change_check,axis=1)
year_over_year_clean.head()

NameError: name 'year_over_year_clean' is not defined

Then we can filter the data by the new column, then take out only the player information and dropa ny duplicates for players that played in multiple teams in one eyar or the other.

In [28]:
y_o_y_changes = year_over_year_clean[year_over_year_clean['CHANGE_TEAM']]
y_o_y_change_players = y_o_y_changes[['PLAYER_ID','PLAYER_NAME']].drop_duplicates()
y_o_y_change_players.head()

NameError: name 'year_over_year_clean' is not defined

and we can check how many players that is

In [29]:
y_o_y_change_players.shape

NameError: name 'y_o_y_change_players' is not defined

### in year_over_year, where suffixes=('_2018','_2019'), why is there an underscore?
```{toggle}
this is optional, but is best practice for pythonic variable naming. (and remember columns are like variables)
```


### If we merge two dataframes which each have a column (or columns) that are the same, but each have different data types in those columns, what happens?

```{toggle}
If they're just named the same, but not the "on" column, you'll get the two columsn with different suffixes default `('_x','_y')`. If it's the `on` column, the merge will be empty
```


### how does suffix to know to change the date?
```{toggle}
Suffix appends what we tell it to to the column names that occur in both datasets
```

## More Practice

1. Use a merge to figure out how many players did not return for the 2019 season
1. Also load the `conferences.csv` data. Use this to figure out how many players moved conferences from 2018 to 2019.